
# POC — Adsorção (CO₂⁻·) Forte vs Fraca em Metais (111): Clássico vs Quântico (AWS Braket)

**Objetivo:** POC mínima, executável hoje em NISQ, que classifica **forte** vs **fraca** adsorção de CO₂⁻· em superfícies (111) de metais, conectando diretamente ao artigo *Amaral et al., JES 2022*.
- **Features (3):** proxy de *Mendeleev* (usaremos **número atômico** como *proxy* nesta POC), **temperatura de fusão** (K) e **raio covalente** (Å).  
- **Classes:** `1 = liga forte` (Rh, Ir, Ni, Pd, Pt) vs `0 = liga fraca` (Cu, Ag, Au, Pb).  
- **Pipelines:** baseline clássico (scikit-learn) e **Quantum Kernel SVM** (feature map quântico em 3 qubits) pelo **AWS Braket (SV1/DM1/QPUs)**.



## 0) Setup de ambiente (AWS Braket + Python)
> Execute esta célula apenas **na instância/notebook AWS** com permissão ao Braket.


In [ ]:

# %pip install -q amazon-braket-sdk amazon-braket-default-simulator boto3 botocore
# %pip install -q scikit-learn pandas numpy matplotlib

import json, math, itertools, time, random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

# AWS Braket SDK
from braket.aws import AwsDevice
from braket.circuits import Circuit
from braket.devices import LocalSimulator

print("Libs ok.")



## 1) Dataset mínimo
- Metais: {Rh, Ir, Ni, Pd, Pt, Cu, Ag, Au, Pb}  
- **Classe**: `1` (forte) = {Rh, Ir, Ni, Pd, Pt}; `0` (fraca) = {Cu, Ag, Au, Pb}  
- **Features (3)**: 
  - `Z`: número atômico (usado aqui como *proxy* para o "Mendeleev number" da literatura)
  - `Tmelt_K`: temperatura de fusão em kelvin
  - `rcov_A`: raio covalente em angstrom
> Os valores abaixo são aproximados para fins de POC. Em produção, substitua por uma tabela verificada.


In [ ]:

data = [
    ("Rh",   45,  2237, 1.35, 1),
    ("Ir",   77,  2719, 1.36, 1),
    ("Ni",   28,  1728, 1.24, 1),
    ("Pd",   46,  1828, 1.39, 1),
    ("Pt",   78,  2041, 1.36, 1),
    ("Cu",   29,  1357, 1.32, 0),
    ("Ag",   47,  1234, 1.45, 0),
    ("Au",   79,  1337, 1.36, 0),
    ("Pb",   82,   601, 1.44, 0),
]
df = pd.DataFrame(data, columns=["metal","Z","Tmelt_K","rcov_A","y"])
display(df)



### Normalização [0,1] e split Leave-One-Out


In [ ]:

X = df[["Z","Tmelt_K","rcov_A"]].to_numpy(dtype=float)
y = df["y"].to_numpy(dtype=int)

scaler = MinMaxScaler()
Xn = scaler.fit_transform(X)

print("X shape:", Xn.shape, "y shape:", y.shape)



## 2) Baseline Clássico (Logistic Regression e SVM-RBF)


In [ ]:

def loo_eval_clf(clf, X, y):
    loo = LeaveOneOut()
    y_true, y_pred = [], []
    for train_idx, test_idx in loo.split(X):
        clf.fit(X[train_idx], y[train_idx])
        y_pred.append(clf.predict(X[test_idx])[0])
        y_true.append(y[test_idx][0])
    acc = accuracy_score(y_true, y_pred)
    cm = confusion_matrix(y_true, y_pred, labels=[0,1])
    return acc, cm

logreg = LogisticRegression(max_iter=1000)
svm_rbf = SVC(kernel="rbf", C=1.0, gamma="scale")

acc_log, cm_log = loo_eval_clf(logreg, Xn, y)
acc_svm, cm_svm = loo_eval_clf(svm_rbf, Xn, y)

print(f"LogReg LOO acc = {acc_log:.3f}\nConfusion:\n{cm_log}")
print("\n---\n")
print(f"SVM RBF LOO acc = {acc_svm:.3f}\nConfusion:\n{cm_svm}")



## 3) Quantum Kernel SVM (AWS Braket)

**Ideia:** Feature map quântico Phi(x) em 3 qubits (angle encoding) + CZ chain.  
Kernel K(x,x') = |<Phi(x)|Phi(x')>|^2 via circuito U(x) adjoint * U(x').



### 3.1) Dispositivo
- `amazon/sv1` (state-vector, ideal)
- `amazon/dm1` (density-matrix, suporta ruído)
- QPUs (trocar ARN conforme conta/região)


In [ ]:

device_arn = "arn:aws:braket:::device/quantum-simulator/amazon/sv1"
device = AwsDevice(device_arn)
device



### 3.2) Feature map (3 qubits, 2 camadas)
Layer(x) = RY(pi*x0) RY(pi*x1) RY(pi*x2) -> CZ(0,1), CZ(1,2), repete 2x.


In [ ]:

import math
from braket.circuits import Circuit

def feature_map_circuit(x3):
    x0, x1, x2 = map(float, x3)
    c = Circuit()
    # Layer 1
    c.ry(0, math.pi * x0).ry(1, math.pi * x1).ry(2, math.pi * x2)
    c.cz(0,1).cz(1,2)
    # Layer 2
    c.ry(0, math.pi * x0).ry(1, math.pi * x1).ry(2, math.pi * x2)
    c.cz(0,1).cz(1,2)
    return c

def overlap_kernel_element(x, xp):
    # U_dag(x) U(x') on |000>, measure prob of |000>
    Ux = feature_map_circuit(x)
    Uxp = feature_map_circuit(xp)
    c = (Ux.adjoint() + Uxp)
    return c

print(feature_map_circuit(Xn[0]))



### 3.3) Matriz de kernel no Braket
- Em SV1, `shots=None` dá probabilidade exata (ideal).  
- Em DM1/QPU, use `shots` (ex.: 4096) e considere mitigação de leitura.


In [ ]:

def kernel_matrix(Xn, device, shots=None, s3_bucket=None, s3_prefix=None):
    n = len(Xn)
    K = np.zeros((n,n), dtype=float)
    # Diagonal
    for i in range(n):
        if shots is None and "sv1" in device.name.lower():
            K[i,i] = 1.0
        else:
            circ = overlap_kernel_element(Xn[i], Xn[i])
            task = device.run(circ, shots=shots, s3_destination_folder=(s3_bucket, s3_prefix) if s3_bucket else None)
            res = task.result()
            counts = getattr(res, "measurement_counts", None)
            if counts:
                total = sum(counts.values())
                p000 = counts.get("000", 0) / (total if total > 0 else 1)
            else:
                probs = getattr(res, "measured_probability", {}) or {}
                p000 = probs.get("000", 1.0)
            K[i,i] = p000
    # Off-diagonais
    for i in range(n):
        for j in range(i+1, n):
            circ = overlap_kernel_element(Xn[i], Xn[j])
            task = device.run(circ, shots=shots, s3_destination_folder=(s3_bucket, s3_prefix) if s3_bucket else None)
            res = task.result()
            counts = getattr(res, "measurement_counts", None)
            if counts:
                total = sum(counts.values())
                p000 = counts.get("000", 0) / (total if total > 0 else 1)
            else:
                probs = getattr(res, "measured_probability", {}) or {}
                p000 = probs.get("000", 0.0)
            K[i,j] = K[j,i] = p000
    return K

print("Funções de kernel prontas.")



### 3.4) LOO com SVM (kernel pré-computado)


In [ ]:

def loo_eval_qkernel_svm(Xn, y, device, shots=None, s3_bucket=None, s3_prefix=None):
    loo = LeaveOneOut()
    y_true, y_pred = [], []
    for test_idx, in loo.split(Xn):
        train_idx = [i for i in range(len(Xn)) if i not in test_idx]
        K_train = kernel_matrix(Xn[train_idx], device, shots, s3_bucket, s3_prefix)
        # Kernel entre teste e treino
        K_test = np.zeros((1, len(train_idx)))
        for j, idx in enumerate(train_idx):
            circ = overlap_kernel_element(Xn[test_idx[0]], Xn[idx])
            task = device.run(circ, shots=shots, s3_destination_folder=(s3_bucket, s3_prefix) if s3_bucket else None)
            res = task.result()
            counts = getattr(res, "measurement_counts", None)
            if counts:
                total = sum(counts.values())
                p000 = counts.get("000", 0) / (total if total > 0 else 1)
            else:
                probs = getattr(res, "measured_probability", {}) or {}
                p000 = probs.get("000", 0.0)
            K_test[0, j] = p000

        clf = SVC(kernel="precomputed", C=1.0)
        clf.fit(K_train, y[train_idx])
        y_hat = clf.predict(K_test)[0]
        y_pred.append(y_hat)
        y_true.append(y[test_idx[0]])
    acc = accuracy_score(y_true, y_pred)
    cm = confusion_matrix(y_true, y_pred, labels=[0,1])
    return acc, cm

print("Pronto para rodar avaliação quantum-kernel (cuidado com custos).")



## 4) (Opcional) DM1 com ruído e variação de shots
- Altere `device_arn` para `amazon/dm1` e ajuste `shots` para {512, 1024, 4096, 8192}.
- Insira canais de ruído após CZ se desejar (ex.: depolarizing).



## 5) Visualização: matrizes de confusão


In [ ]:

def plot_confusion(cm, title="Confusion Matrix"):
    fig, ax = plt.subplots(figsize=(3.5,3.2))
    im = ax.imshow(cm, interpolation='nearest')
    ax.set_title(title)
    ax.set_xticks([0,1]); ax.set_yticks([0,1])
    ax.set_xticklabels(['weak(0)','strong(1)']); ax.set_yticklabels(['weak(0)','strong(1)'])
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, cm[i,j], ha="center", va="center")
    ax.set_xlabel("Pred"); ax.set_ylabel("True")
    plt.show()

plot_confusion(cm_log, "LogReg LOO")
plot_confusion(cm_svm, "SVM RBF LOO")



## 6) (Opcional) Persistência S3


In [ ]:

S3_BUCKET = None  # ex.: "meu-bucket-braket"
S3_PREFIX = "experimentos/poc_qkernel"



## 7) QPU real (instruções)
- Liste e selecione um QPU disponível na sua conta/região (Rigetti, IonQ, OQC, QuEra etc.).
- Ajuste `shots` (>= 4000) e considere mitigação de erro de leitura.



## 8) Critérios de sucesso para a demo
- Baselines clássicos com acurácia LOO > 0.6.
- Quantum kernel (SV1 ideal) ~ baseline ou melhor.
- Em DM1/QPU, acurácia > 0.6–0.7 com melhora ao aumentar shots/mitigação.
